In [34]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler

in_dir = '../data/'
out_dir = '../prep_data/'
sets = ['Autism-Adult-Data/Autism-Adult-Data-preproc.csv',
        'bankruptcy/bankrupt.csv',
        'breast-cancer/breast-cancer-wisconsin.data',
        'contraceptive/contraceptive.csv',
        'horse-colic/horse-colic.data-preproc.csv',
        'hr-analytics/HR_comma_sep.csv',
        'Qualitative_Bankruptcy/Qualitative_Bankruptcy.data.txt',
        'student/student_grades.csv',
        'teaching-english/tae.csv',
        'website-phishing/PhishingData.csv',
        'wine-quality/winequality-red.csv',
        'covertypes/covertype_scale.csv',
        'amazon/amzreviews.csv',
        'kidney/kidney_colMeanMode.csv',
        'congress/congress_leave.csv'
       ]

In [37]:
scaler = MinMaxScaler()
for i in range(0, len(sets)):
    df = pd.read_csv(in_dir+sets[i])
    df.replace(['?',' '], np.NaN, inplace=True)
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype('category').cat.codes
    df.fillna(df.median(), inplace=True)
    inds=None
    if df.shape[0] > 2000:
        print("Reducing %s from %d to %d samples" % (sets[i], df.shape[0], 2000))
        df = df.sample(n=2000)
        inds = df.index.values
    
    X = df.drop('Class',axis=1)
    Y = df['Class'].copy()
    if df.shape[1] > 100:
        oldShape = df.shape[1]-1
        clf = ExtraTreesClassifier()
        clf = clf.fit(X, Y)
        model = SelectFromModel(clf, prefit=True)
        X = model.transform(X)
        print("Reducing %s from %d to %d features" % (sets[i], oldShape, X.shape[1]))
        print("Class label is now at index %d" % (X.shape[1]))
    X = scaler.fit_transform(X)
    df = pd.DataFrame(X, index=inds, columns=list(range(0,X.shape[1])))
    df = df.join(Y)
    df.to_csv(out_dir+sets[i], index = False)
    

Reducing hr-analytics/HR_comma_sep.csv from 14999 to 2000 samples
Reducing covertypes/covertype_scale.csv from 5814 to 2000 samples
Reducing amazon/amzreviews.csv from 10001 to 3093 features
Class label is now at index 3093


C:\Users\Adi\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
